In [ ]:
library(tidyverse)
library(readxl)

# 1. NAČTENÍ DAT
data <- read_excel("portfolia_vynosy.xlsx")

# --- 🔧 OPRAVA DESETINNÝCH ČÁREK (POJISTKA) ---
# Pokud Rko načetlo výnosy jako text (kvůli čárce "15,2"), musíme to opravit.
# Pokud už to jsou čísla, tento krok nic nezkazí.
data$Rocni_Vynosnost_v_Procentech <- as.numeric(gsub(",", ".", data$Rocni_Vynosnost_v_Procentech))

# Rychlá kontrola - musí to psát "numeric" nebo "double"
print(class(data$Rocni_Vynosnost_v_Procentech))


# ==============================================================================
# 🅰️ TEST PRO STRATEGII A
# ==============================================================================

# 1. Filtrace dat pro A
data_A <- data %>%
  filter(Strategie == "A") %>%
  drop_na(Rocni_Vynosnost_v_Procentech) # Vyhodíme prázdné řádky

# 2. Počty (n a x)
n_A <- nrow(data_A)                                   # Počet všech portfolií A
x_A <- sum(data_A$Rocni_Vynosnost_v_Procentech > 0)   # Počet ziskových (> 0)

# Výpis pro kontrolu
print(paste("Strategie A -> Ziskových:", x_A, "z celkových", n_A))

# 3. Test (H0: p <= 0.7 vs H1: p > 0.7)
# Hladina významnosti 10% -> conf.level = 0.90
test_A <- prop.test(x = x_A, n = n_A, p = 0.70, alternative = "greater", conf.level = 0.90)

print("--- VÝSLEDEK A ---")
print(test_A)
print(paste("P-hodnota A:", format.pval(test_A$p.value, digits = 4)))


# ==============================================================================
# 🅱️ TEST PRO STRATEGII B
# ==============================================================================

# 1. Filtrace dat pro B
data_B <- data %>%
  filter(Strategie == "B") %>%
  drop_na(Rocni_Vynosnost_v_Procentech)

# 2. Počty
n_B <- nrow(data_B)
x_B <- sum(data_B$Rocni_Vynosnost_v_Procentech > 0)

print(paste("Strategie B -> Ziskových:", x_B, "z celkových", n_B))

# 3. Test
test_B <- prop.test(x = x_B, n = n_B, p = 0.70, alternative = "greater", conf.level = 0.90)

print("--- VÝSLEDEK B ---")
print(test_B)
print(paste("P-hodnota B:", format.pval(test_B$p.value, digits = 4)))

[1] "numeric"
[1] "Strategie A -> Ziskových: 37 z celkových 50"
[1] "--- VÝSLEDEK A ---"

	1-sample proportions test with continuity correction

data:  x_A out of n_A, null probability 0.7
X-squared = 0.21429, df = 1, p-value = 0.3217
alternative hypothesis: true p is greater than 0.7
90 percent confidence interval:
 0.6431753 1.0000000
sample estimates:
   p 
0.74 

[1] "P-hodnota A: 0.3217"
[1] "Strategie B -> Ziskových: 39 z celkových 45"
[1] "--- VÝSLEDEK B ---"

	1-sample proportions test with continuity correction

data:  x_B out of n_B, null probability 0.7
X-squared = 5.1852, df = 1, p-value = 0.01139
alternative hypothesis: true p is greater than 0.7
90 percent confidence interval:
 0.7758921 1.0000000
sample estimates:
        p 
0.8666667 

[1] "P-hodnota B: 0.01139"


In [4]:
# --- VÝPOČET BODOVÉHO ODHADU pA ---
data_A <- data %>% filter(Strategie == "A")

# Počet úspěšných (kde je výnos > 0) děleno všemi v A
p_A <- sum(data_A$Rocni_Vynosnost_v_Procentech > 0, na.rm = TRUE) / nrow(data_A)


# --- VÝPOČET BODOVÉHO ODHADU pB ---
data_B <- data %>% filter(Strategie == "B")

# Počet úspěšných děleno všemi v B
p_B <- sum(data_B$Rocni_Vynosnost_v_Procentech > 0, na.rm = TRUE) / nrow(data_B)


# --- VÝPIS VÝSLEDKŮ ---
print(paste("pA =", round(p_A, 4))) # Zaokrouhlujeme na 4 místa
print(paste("pB =", round(p_B, 4)))

[1] "pA = 0.74"
[1] "pB = 0.8667"


In [5]:


# --- 1. Definice funkce pro výpočet minimálního n ---
# Vzorec ze zadání: n > 9 / (p * (1 - p))
calc_min_n <- function(p) {
  hodnota <- 9 / (p * (1 - p))
  return(ceiling(hodnota)) # Zaokrouhlujeme VŽDY NAHORU na celé číslo
}

# --- 2. Výpočet pro Strategii A ---
# (Používáme p_A a n_A z předchozího kódu)
# Pokud je nemáš v paměti, znovu spusť kód pro bodové odhady
limit_A <- calc_min_n(p_A)

print("--- STRATEGIE A ---")
print(paste("Bodový odhad pA:", round(p_A, 4)))
print(paste("Minimální nutný rozsah (Odpověď 4):", limit_A))
print(paste("Skutečný rozsah (n_A):", n_A))

if(n_A > limit_A) {
  print("✅ Předpoklad JE splněn.")
} else {
  print("❌ Předpoklad NENÍ splněn.")
}

# --- 3. Výpočet pro Strategii B ---
limit_B <- calc_min_n(p_B)

print("--- STRATEGIE B ---")
print(paste("Bodový odhad pB:", round(p_B, 4)))
print(paste("Minimální nutný rozsah (Odpověď 5):", limit_B))
print(paste("Skutečný rozsah (n_B):", n_B))

if(n_B > limit_B) {
  print("✅ Předpoklad JE splněn.")
} else {
  print("❌ Předpoklad NENÍ splněn.")
}

[1] "--- STRATEGIE A ---"
[1] "Bodový odhad pA: 0.74"
[1] "Minimální nutný rozsah (Odpověď 4): 47"
[1] "Skutečný rozsah (n_A): 50"
[1] "✅ Předpoklad JE splněn."
[1] "--- STRATEGIE B ---"
[1] "Bodový odhad pB: 0.8667"
[1] "Minimální nutný rozsah (Odpověď 5): 78"
[1] "Skutečný rozsah (n_B): 45"
[1] "❌ Předpoklad NENÍ splněn."


In [6]:

# Předpoklad: Máš načtená data a spočítané x_A, n_A, x_B, n_B z minula.
# Pokud ne, spusť si znovu ten blok "Výpočet metrik" nebo "Načtení".

# ==========================================
# 🅰️ INTERVAL PRO STRATEGII A
# ==========================================
# conf.level = 0.90 (Zadání: 90%)
# alternative = "greater" (Protože testujeme, zda je to VĚTŠÍ)

ci_A <- prop.test(x = x_A, n = n_A, conf.level = 0.90, alternative = "greater")

print("--- 90% INTERVALOVÝ ODHAD (Strategie A) ---")
print(ci_A$conf.int)
# Výstup bude ve formátu: [Dolní_mez, 1.0]


# ==========================================
# 🅱️ INTERVAL PRO STRATEGII B
# ==========================================

ci_B <- prop.test(x = x_B, n = n_B, conf.level = 0.90, alternative = "greater")

print("--- 90% INTERVALOVÝ ODHAD (Strategie B) ---")
print(ci_B$conf.int)

[1] "--- 90% INTERVALOVÝ ODHAD (Strategie A) ---"
[1] 0.6431753 1.0000000
attr(,"conf.level")
[1] 0.9
[1] "--- 90% INTERVALOVÝ ODHAD (Strategie B) ---"
[1] 0.7758921 1.0000000
attr(,"conf.level")
[1] 0.9


In [7]:
# ==========================================
# 🏁 GENERÁTOR ZÁVĚRU
# ==========================================

# 1. ROZHODNUTÍ O STRATEGII (Odpověď 12)
# Podmínka: n musí být větší než limit. Pokud obě splňují, bereme A.
splneno_A <- n_A > limit_A
splneno_B <- n_B > limit_B

vybrana_strategie <- ""

if(splneno_A && splneno_B) {
  vybrana_strategie <- "A" # Zadání: "Pokud splněno pro obě, interpretujte A"
} else if (splneno_A) {
  vybrana_strategie <- "A"
} else if (splneno_B) {
  vybrana_strategie <- "B"
} else {
  vybrana_strategie <- "CHYBA - Ani jedna nesplňuje"
}

# 2. NAČTENÍ DAT PRO VYBRANOU STRATEGII
if(vybrana_strategie == "A") {
  p_hodnota <- test_A$p.value
} else {
  p_hodnota <- test_B$p.value
}

# 3. ROZHODNUTÍ (Odpověď 14)
# Hladina významnosti 0.10 (10 %)
# Pokud p < 0.10 -> ZAMÍTÁME H0 -> PŘEVYŠUJE
# Pokud p >= 0.10 -> NEZAMÍTÁME H0 -> NEPŘEVYŠUJE

sloveso <- ""
if(p_hodnota < 0.10) {
  sloveso <- "převyšuje"       # Výsledek je statisticky významný
} else {
  sloveso <- "nepřevyšuje"     # Neprokázalo se to
}

# Ošetření formátu p-hodnoty (pokud je super malá)
p_hodnota_vypis <- format.pval(p_hodnota, digits = 4)
if(p_hodnota < 0.001) { p_hodnota_vypis <- "0.001" }


# ==========================================
# 📝 VÝPIS ODPOVĚDÍ
# ==========================================
print(paste("Odpověď 12 (Strategie):", vybrana_strategie))
print(paste("Odpověď 13 (p-hodnota):", p_hodnota_vypis))
print(paste("Odpověď 14 (Závěr):", sloveso))

[1] "Odpověď 12 (Strategie): A"
[1] "Odpověď 13 (p-hodnota): 0.3217"
[1] "Odpověď 14 (Závěr): nepřevyšuje"


In [8]:
library(tidyverse)
library(readxl)

# 1. NAČTENÍ A PŘÍPRAVA
data <- read_excel("portfolia_vynosy.xlsx")

# Pojistka: Oprava desetinných čárek
if(is.character(data$Rocni_Vynosnost_v_Procentech)) {
  data$Rocni_Vynosnost_v_Procentech <- as.numeric(gsub(",", ".", data$Rocni_Vynosnost_v_Procentech))
}

# 2. VYTVOŘENÍ NOVÉ PROMĚNNÉ (Kategorizace)
# Pokud je výnos > 0 -> "Zisk", jinak "Ztráta"
data <- data %>%
  mutate(vysledek = ifelse(Rocni_Vynosnost_v_Procentech > 0, "Zisk", "Ztrata"))

# 3. KONTINGENČNÍ TABULKA (Pozorované četnosti)
# Řádky = Strategie, Sloupce = Výsledek
tabulka <- table(data$Strategie, data$vysledek)

print("--- POZOROVANÉ ČETNOSTI (Kolik jich skutečně je) ---")
print(tabulka)

# 4. PROVEDENÍ TESTU (Chí-kvadrát)
# correct = FALSE (pro klasický Pearsonův test, někdy se používá TRUE pro Yatesovu korekci)
test_chi <- chisq.test(tabulka, correct = FALSE) 

print("--- VÝSLEDEK CHÍ-KVADRÁT TESTU ---")
print(test_chi)

# 5. KONTROLA PŘEDPOKLADŮ (Očekávané četnosti)
# Aby test platil, měly by být očekávané četnosti ideálně > 5
print("--- OČEKÁVANÉ ČETNOSTI (Theoretical counts) ---")
print(test_chi$expected)

[1] "--- POZOROVANÉ ČETNOSTI (Kolik jich skutečně je) ---"
   
    Zisk Ztrata
  A   37     13
  B   39      6
[1] "--- VÝSLEDEK CHÍ-KVADRÁT TESTU ---"

	Pearson's Chi-squared test

data:  tabulka
X-squared = 2.375, df = 1, p-value = 0.1233

[1] "--- OČEKÁVANÉ ČETNOSTI (Theoretical counts) ---"
   
    Zisk Ztrata
  A   40     10
  B   36      9


In [ ]:
library(tidyverse)
library(readxl)
#install.packages("vcd") # Odkomentuj a spusť, pokud nemáš balíček 'vcd' (pro Cramerovo V)
library(vcd) 

# 1. NAČTENÍ A PŘÍPRAVA DAT
data <- read_excel("portfolia_vynosy.xlsx")

# Oprava desetinných čárek (pro jistotu)
if(is.character(data$Rocni_Vynosnost_v_Procentech)) {
  data$Rocni_Vynosnost_v_Procentech <- as.numeric(gsub(",", ".", data$Rocni_Vynosnost_v_Procentech))
}

# Vytvoření proměnné Vynos (Zisk/Ztrata)
data <- data %>%
  mutate(Vysledek = ifelse(Rocni_Vynosnost_v_Procentech > 0, "Zisk", "Ztrata"))

# 2. TABULKA ČETNOSTÍ
# Důležité: Aby Rko počítalo Odds Ratio správně, musí být tabulka ve formátu:
#           | Zisk | Ztráta
# Strat A   |  a   |   b
# Strat B   |  c   |   d
# Proto používáme factor s explicitním pořadím levelů
data$Vysledek <- factor(data$Vysledek, levels = c("Zisk", "Ztrata"))
data$Strategie <- factor(data$Strategie) # A bude první, B druhé

tabulka <- table(data$Strategie, data$Vysledek)
print("--- KONTINGENČNÍ TABULKA ---")
print(tabulka)

# Hodnoty z tabulky pro ruční výpočet (pro pochopení)
# [1,1] = A Zisk, [1,2] = A Ztráta
# [2,1] = B Zisk, [2,2] = B Ztráta
a <- tabulka[1,1]; b <- tabulka[1,2]
c <- tabulka[2,1]; d <- tabulka[2,2]


# ==========================================
# 1. ČÁST: ŠANCE (ODDS)
# ==========================================

# Šance pro strategii A = Počet Ziskových A / Počet Ztrátových A
odds_A <- a / b
# Šance pro strategii B = Počet Ziskových B / Počet Ztrátových B
odds_B <- c / d

print(paste("Odpověď 9 (Šance A):", round(odds_A, 2)))
print(paste("Odpověď 10 (Na 100 ztrátových A):", round(odds_A * 100, 2))) # nebo zaokrouhli na celé číslo dle kontextu

print(paste("Odpověď 11 (Šance B):", round(odds_B, 2)))
print(paste("Odpověď 12 (Na 100 ztrátových B):", round(odds_B * 100, 2)))


# ==========================================
# 2. ČÁST: POMĚR ŠANCÍ (ODDS RATIO)
# ==========================================

# Odds Ratio (OR) = Šance A / Šance B
OR <- odds_A / odds_B

print("--- POMĚR ŠANCÍ (OR) ---")
# Zjištění, která strategie je lepší (pro doplnění textu)
lepsi_strategie <- ifelse(odds_A > odds_B, "A", "B")
horsi_strategie <- ifelse(odds_A > odds_B, "B", "A")
hodnota_OR_text <- ifelse(odds_A > odds_B, OR, 1/OR) # Abychom měli číslo > 1

print(paste("Odpověď 13 (Lepší strategie):", lepsi_strategie))
print(paste("Odpověď 14 (Kolikrát větší šance - OR):", round(hodnota_OR_text, 2)))
print(paste("Odpověď 15 (Srovnávaná strategie):", horsi_strategie))

# Intervalový odhad pro Odds Ratio (využijeme funkci loddsratio z vcd nebo fisher.test)
# fisher.test počítá OR a jeho interval spolehlivosti nejpřesněji
test_fisher <- fisher.test(tabulka, conf.level = 0.95)

print(paste("Odpověď 16 (Dolní mez OR):", round(test_fisher$conf.int[1], 4)))
print(paste("Odpověď 17 (Horní mez OR):", round(test_fisher$conf.int[2], 4)))


# ==========================================
# 3. ČÁST: CRAMEROVO V (Síla závislosti)
# ==========================================

# Výpočet Cramerova V
assoc_stats <- assocstats(tabulka)
cramer_v <- assoc_stats$cramer

print(paste("Odpověď 21 (Cramerovo V):", round(cramer_v, 3)))

# Slovní interpretace (orientační)
interpretace <- case_when(
  cramer_v < 0.1 ~ "zanedbatelná",
  cramer_v < 0.3 ~ "slabá",
  cramer_v < 0.5 ~ "středně silná",
  TRUE ~ "silná"
)
print(paste("Odpověď 22 (Interpretace):", interpretace))

[1] "--- KONTINGENČNÍ TABULKA ---"
   
    Zisk Ztrata
  A   37     13
  B   39      6
[1] "Odpověď 9 (Šance A): 2.85"
[1] "Odpověď 10 (Na 100 ztrátových A): 284.62"
[1] "Odpověď 11 (Šance B): 6.5"
[1] "Odpověď 12 (Na 100 ztrátových B): 650"
[1] "--- POMĚR ŠANCÍ (OR) ---"
[1] "Odpověď 13 (Lepší strategie): B"
[1] "Odpověď 14 (Kolikrát větší šance - OR): 2.28"
[1] "Odpověď 15 (Srovnávaná strategie): A"
[1] "Odpověď 16 (Dolní mez OR): 0.124"
[1] "Odpověď 17 (Horní mez OR): 1.4054"
[1] "Odpověď 21 (Cramerovo V): 0.158"
[1] "Odpověď 22 (Interpretace): slabá"
